# 🎰 Lecture 4: Lottery Ticket Hypothesis - Complete Demo

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gaurav-redhat/efficientml_course/blob/main/04_pruning_sparsity_2/demo.ipynb)

## What You'll Learn
- The Lottery Ticket Hypothesis (LTH)
- Finding winning tickets through iterative pruning
- Why initialization matters for sparse networks
- Practical implementation of LTH

In [ ]:
!pip install torch torchvision matplotlib numpy -q
import torch
import torch.nn as nn
import torch.nn.utils.prune as prune
import matplotlib.pyplot as plt
import numpy as np
import copy

torch.manual_seed(42)
print('Ready for Lottery Ticket discovery!')

## Part 1: The Lottery Ticket Hypothesis Explained

> **"A randomly-initialized, dense neural network contains a subnetwork that, 
> when trained in isolation, can match the test accuracy of the original network 
> after training for at most the same number of iterations."**
> 
> — Frankle & Carlin, 2019

🎰 Think of it like a lottery:
- **Dense network** = Buying ALL lottery tickets
- **Winning ticket** = The sparse subnetwork that can win (achieve good accuracy)
- **Key insight** = The winning ticket was there from the START (initialization)

In [ ]:
# Visualization of the hypothesis
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

# Dense network
dense = np.random.randn(20, 20) * 0.1
axes[0].imshow(dense, cmap='RdBu', vmin=-0.3, vmax=0.3)
axes[0].set_title('1. Dense Network\n(Random Init)', fontsize=12)
axes[0].axis('off')

# Training
trained = dense + np.random.randn(20, 20) * 0.3
trained[np.abs(trained) < 0.15] *= 0.1  # Many weights become small
axes[1].imshow(trained, cmap='RdBu', vmin=-0.5, vmax=0.5)
axes[1].set_title('2. After Training\n(Many small weights)', fontsize=12)
axes[1].axis('off')

# Pruning
mask = np.abs(trained) > 0.2
pruned = trained * mask
axes[2].imshow(pruned, cmap='RdBu', vmin=-0.5, vmax=0.5)
axes[2].set_title('3. After Pruning\n(Sparse network)', fontsize=12)
axes[2].axis('off')

# Winning ticket (original init with mask)
winning_ticket = dense * mask
axes[3].imshow(winning_ticket, cmap='RdBu', vmin=-0.3, vmax=0.3)
axes[3].set_title('4. WINNING TICKET\n(Original init + mask)', fontsize=12)
axes[3].axis('off')

plt.suptitle('🎰 The Lottery Ticket Hypothesis', fontsize=14)
plt.tight_layout()
plt.show()

print('💡 The winning ticket uses the ORIGINAL initialization, not trained weights!')

## Part 2: Implementing Lottery Ticket Discovery

In [ ]:
class LotteryNet(nn.Module):
    """Simple network for lottery ticket experiments."""
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 300)
        self.fc2 = nn.Linear(300, 100)
        self.fc3 = nn.Linear(100, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return self.fc3(x)

def save_initial_weights(model):
    """Save initial weights for later rewinding."""
    return {name: param.data.clone() for name, param in model.named_parameters()}

def rewind_weights(model, initial_weights, masks):
    """Rewind to initial weights with pruning mask applied."""
    for name, param in model.named_parameters():
        if name in masks:
            param.data = initial_weights[name] * masks[name]
        else:
            param.data = initial_weights[name].clone()

def create_masks(model, sparsity):
    """Create pruning masks based on trained weight magnitudes."""
    masks = {}
    for name, param in model.named_parameters():
        if 'weight' in name:
            threshold = torch.quantile(param.data.abs().flatten(), sparsity)
            masks[name] = (param.data.abs() > threshold).float()
    return masks

def apply_masks(model, masks):
    """Apply masks to model weights."""
    for name, param in model.named_parameters():
        if name in masks:
            param.data *= masks[name]

def count_nonzero(model):
    """Count non-zero parameters."""
    total = sum(p.numel() for p in model.parameters())
    nonzero = sum((p != 0).sum().item() for p in model.parameters())
    return total, nonzero

# Create synthetic data
X_train = torch.randn(2000, 784)
y_train = torch.randint(0, 10, (2000,))
X_test = torch.randn(500, 784)
y_test = torch.randint(0, 10, (500,))

print('✅ Lottery Ticket utilities ready!')

## Part 3: Finding the Winning Ticket

In [ ]:
def train_model(model, X, y, masks=None, epochs=10, lr=0.01):
    """Train model with optional mask application."""
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    losses = []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X)
        loss = criterion(outputs, y)
        
        # Backward pass
        loss.backward()
        
        # Zero gradients of pruned weights
        if masks:
            for name, param in model.named_parameters():
                if name in masks and param.grad is not None:
                    param.grad.data *= masks[name]
        
        optimizer.step()
        
        # Re-apply masks
        if masks:
            apply_masks(model, masks)
        
        losses.append(loss.item())
    
    return losses

def evaluate(model, X, y):
    """Evaluate model accuracy."""
    model.eval()
    with torch.no_grad():
        outputs = model(X)
        _, predicted = outputs.max(1)
        accuracy = (predicted == y).float().mean().item() * 100
    return accuracy

# Initialize model and save initial weights
model = LotteryNet()
initial_weights = save_initial_weights(model)

print('🎰 LOTTERY TICKET EXPERIMENT')
print('=' * 60)

# Step 1: Train original dense network
print('\n1️⃣ Training dense network...')
dense_losses = train_model(model, X_train, y_train, epochs=20)
dense_acc = evaluate(model, X_test, y_test)
print(f'   Dense accuracy: {dense_acc:.1f}%')

# Step 2: Create masks from trained network
print('\n2️⃣ Creating pruning masks (80% sparsity)...')
masks = create_masks(model, sparsity=0.8)

# Step 3: Random reinitialization (baseline)
print('\n3️⃣ Training with random reinitialization...')
random_model = LotteryNet()  # New random init
apply_masks(random_model, masks)  # Apply same mask
random_losses = train_model(random_model, X_train, y_train, masks=masks, epochs=20)
random_acc = evaluate(random_model, X_test, y_test)
print(f'   Random sparse accuracy: {random_acc:.1f}%')

# Step 4: Winning ticket (original init + mask)
print('\n4️⃣ Training WINNING TICKET (original init + mask)...')
winning_model = LotteryNet()
rewind_weights(winning_model, initial_weights, masks)
winning_losses = train_model(winning_model, X_train, y_train, masks=masks, epochs=20)
winning_acc = evaluate(winning_model, X_test, y_test)
print(f'   Winning ticket accuracy: {winning_acc:.1f}%')

# Count parameters
total, nonzero = count_nonzero(winning_model)
print(f'\n📊 Parameters: {nonzero:,} / {total:,} ({100*nonzero/total:.1f}% remaining)')

In [ ]:
# Visualize results
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Training curves
axes[0].plot(dense_losses, label=f'Dense ({dense_acc:.1f}%)', linewidth=2, color='#3b82f6')
axes[0].plot(random_losses, label=f'Random Sparse ({random_acc:.1f}%)', linewidth=2, color='#ef4444')
axes[0].plot(winning_losses, label=f'Winning Ticket ({winning_acc:.1f}%)', linewidth=2, color='#22c55e')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('📈 Training Curves')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy comparison
categories = ['Dense\n(100%)', 'Random Sparse\n(20%)', 'Winning Ticket\n(20%)']
accuracies = [dense_acc, random_acc, winning_acc]
colors = ['#3b82f6', '#ef4444', '#22c55e']

bars = axes[1].bar(categories, accuracies, color=colors)
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('📊 Accuracy Comparison')
for bar, acc in zip(bars, accuracies):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                 f'{acc:.1f}%', ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print('\n🎯 KEY RESULT: Winning ticket (20% weights) matches dense network!')

## Part 4: Iterative Magnitude Pruning (IMP)

In [ ]:
def iterative_magnitude_pruning(prune_rate=0.2, iterations=10):
    """
    Iterative Magnitude Pruning:
    1. Train network
    2. Prune p% of smallest weights
    3. Reset to initial weights (with mask)
    4. Repeat
    """
    model = LotteryNet()
    initial_weights = save_initial_weights(model)
    
    # Track cumulative mask
    cumulative_mask = {name: torch.ones_like(param) 
                       for name, param in model.named_parameters() 
                       if 'weight' in name}
    
    results = []
    
    for iteration in range(iterations):
        # Reset to initial weights with current mask
        rewind_weights(model, initial_weights, cumulative_mask)
        
        # Train
        train_model(model, X_train, y_train, masks=cumulative_mask, epochs=15)
        
        # Evaluate
        acc = evaluate(model, X_test, y_test)
        total, nonzero = count_nonzero(model)
        sparsity = 100 * (1 - nonzero/total)
        
        results.append({'iteration': iteration, 'accuracy': acc, 'sparsity': sparsity})
        print(f'Iteration {iteration}: Sparsity={sparsity:.1f}%, Accuracy={acc:.1f}%')
        
        # Prune for next iteration
        for name, param in model.named_parameters():
            if name in cumulative_mask:
                # Only consider non-zero weights for pruning
                nonzero_weights = param.data * cumulative_mask[name]
                if nonzero_weights.abs().sum() > 0:
                    # Find threshold for this layer
                    nonzero_abs = nonzero_weights.abs()[cumulative_mask[name] == 1]
                    if len(nonzero_abs) > 0:
                        threshold = torch.quantile(nonzero_abs.flatten(), prune_rate)
                        # Update cumulative mask
                        cumulative_mask[name] *= (nonzero_weights.abs() > threshold).float()
    
    return results

print('🔄 ITERATIVE MAGNITUDE PRUNING')
print('=' * 60)
imp_results = iterative_magnitude_pruning(prune_rate=0.2, iterations=8)

In [ ]:
# Visualize IMP results
fig, ax = plt.subplots(figsize=(10, 6))

sparsities = [r['sparsity'] for r in imp_results]
accuracies = [r['accuracy'] for r in imp_results]

ax.plot(sparsities, accuracies, 'o-', linewidth=2, markersize=10, color='#22c55e')
ax.axhline(y=accuracies[0], color='#3b82f6', linestyle='--', alpha=0.7, label='Dense baseline')

# Mark the "knee" point
for i, (s, a) in enumerate(zip(sparsities, accuracies)):
    ax.annotate(f'Iter {i}', (s, a), xytext=(5, 5), textcoords='offset points', fontsize=9)

ax.set_xlabel('Sparsity (%)', fontsize=12)
ax.set_ylabel('Accuracy (%)', fontsize=12)
ax.set_title('📈 Iterative Magnitude Pruning Results', fontsize=14)
ax.grid(True, alpha=0.3)
ax.legend()

plt.tight_layout()
plt.show()

print('\n💡 IMP finds winning tickets at higher sparsity levels!')

## Part 5: Late Rewinding (Practical Improvement)

In [ ]:
def late_rewinding_experiment():
    """
    Late Rewinding: Instead of rewinding to iteration 0,
    rewind to iteration k (e.g., after 10% of training).
    
    This often works better for larger models!
    """
    model = LotteryNet()
    
    # Save checkpoints at different iterations
    checkpoints = {}
    checkpoint_epochs = [0, 2, 5, 10]
    
    optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    
    # Train and save checkpoints
    for epoch in range(20):
        if epoch in checkpoint_epochs:
            checkpoints[epoch] = {name: p.data.clone() for name, p in model.named_parameters()}
        
        model.train()
        optimizer.zero_grad()
        loss = criterion(model(X_train), y_train)
        loss.backward()
        optimizer.step()
    
    # Create pruning mask from final trained model
    masks = create_masks(model, sparsity=0.8)
    
    # Test different rewind points
    print('📊 LATE REWINDING EXPERIMENT')
    print('=' * 50)
    print(f'{"Rewind to Epoch":<20} {"Accuracy":<15}')
    print('-' * 35)
    
    results = []
    for rewind_epoch, weights in checkpoints.items():
        test_model = LotteryNet()
        rewind_weights(test_model, weights, masks)
        train_model(test_model, X_train, y_train, masks=masks, epochs=20)
        acc = evaluate(test_model, X_test, y_test)
        results.append((rewind_epoch, acc))
        print(f'{rewind_epoch:<20} {acc:.1f}%')
    
    return results

late_rewind_results = late_rewinding_experiment()

# Visualize
fig, ax = plt.subplots(figsize=(8, 5))
epochs = [r[0] for r in late_rewind_results]
accs = [r[1] for r in late_rewind_results]

ax.bar([str(e) for e in epochs], accs, color='#8b5cf6')
ax.set_xlabel('Rewind Epoch')
ax.set_ylabel('Final Accuracy (%)')
ax.set_title('📊 Late Rewinding: Different Rewind Points')

for i, (e, a) in enumerate(zip(epochs, accs)):
    ax.text(i, a + 0.5, f'{a:.1f}%', ha='center')

plt.tight_layout()
plt.show()

print('\n💡 Late rewinding often works better than iteration 0!')

In [ ]:
print('🎯 KEY TAKEAWAYS')
print('=' * 60)
print('\n1. Lottery Ticket Hypothesis: Sparse winning tickets exist at init')
print('\n2. Winning tickets: Original init + pruning mask from trained net')
print('\n3. Random sparse != Winning ticket (init matters!)')
print('\n4. Iterative Magnitude Pruning finds tickets at higher sparsity')
print('\n5. Late Rewinding: Rewind to early training, not init')
print('\n6. LTH inspired efficient training and sparse architecture search')
print('\n' + '=' * 60)
print('\n📚 Next: Quantization for further compression!')